In [2]:
import pandas as pd
import sqlite3 as sql

In [3]:
db = sql.connect('./Products.db')

In [8]:
query = 'SELECT * FROM Customers;'

#make cursor
cur = db.cursor()
cur.execute(query)
cur.fetchall()[0]

('1',
 'Alfreds Futterkiste',
 'Maria Anders',
 'Obere Str. 57',
 'Berlin',
 '12209',
 'Germany')

In [13]:
df = pd.read_sql(query, db)
df.head()

,CustomerID,CustomerName,ContactName,Address,City,PostalCode,Country
0,1,Alfreds Futterkiste,Maria Anders,Obere Str. 57,Berlin,12209,Germany
1,2,Ana Trujillo Emparedados y helados,Ana Trujillo,Avda. de la Constitución 2222,México D.F.,05021,Mexico
2,3,Antonio Moreno Taquería,Antonio Moreno,Mataderos 2312,México D.F.,05023,Mexico
3,4,Around the Horn,Thomas Hardy,120 Hanover Sq.,London,WA1 1DP,UK
4,5,Berglunds snabbköp,Christina Berglund,Berguvsvägen 8,Luleå,S-958 22,Sweden


In [35]:
query = """
SELECT tbl_name
    FROM sqlite_master
;
"""
pd.read_sql(query,db)

,tbl_name
0,Categories
1,Customers
2,Employees
3,Shippers
4,Suppliers
5,Orders
6,OrderDetails
7,Products


In [52]:
query = """
SELECT * 
FROM Customers
;
"""
df = pd.read_sql(query,db)

In [56]:
pd.options.display.max_colwidth = 300
query = """
SELECT sql
    FROM sqlite_master
;
"""
pd.read_sql(query,db)
df[(df["Country"]=="Germany") | (df["City"]=="Paris")]

,CustomerID,CustomerName,ContactName,Address,City,PostalCode,Country
0,1,Alfreds Futterkiste,Maria Anders,Obere Str. 57,Berlin,12209,Germany
5,6,Blauer See Delikatessen,Hanna Moos,Forsterstr. 57,Mannheim,68306,Germany
16,17,Drachenblut Delikatessend,Sven Ottlieb,Walserweg 21,Aachen,52066,Germany
24,25,Frankenversand,Peter Franken,Berliner Platz 43,München,80805,Germany
38,39,Königlich Essen,Philip Cramer,Maubelstr. 90,Brandenburg,14776,Germany
43,44,Lehmanns Marktstand,Renate Messner,Magazinweg 7,Frankfurt a.M.,60528,Germany
51,52,Morgenstern Gesundkost,Alexander Feuer,Heerstr. 22,Leipzig,04179,Germany
55,56,Ottilies Käseladen,Henriette Pfalzheim,Mehrheimerstr. 369,Köln,50739,Germany
56,57,Paris spécialités,Marie Bertrand,265 boulevard Charonne,Paris,75012,France
62,63,QUICK-Stop,Horst Kloss,Taucherstraße 10,Cunewalde,01307,Germany


In [57]:
query = '''
SELECT * 
FROM Customers 
WHERE Country == 'Germany' OR City == "Paris"
;
'''
df = pd.read_sql(query,db)
df

,CustomerID,CustomerName,ContactName,Address,City,PostalCode,Country
0,1,Alfreds Futterkiste,Maria Anders,Obere Str. 57,Berlin,12209,Germany
1,6,Blauer See Delikatessen,Hanna Moos,Forsterstr. 57,Mannheim,68306,Germany
2,17,Drachenblut Delikatessend,Sven Ottlieb,Walserweg 21,Aachen,52066,Germany
3,25,Frankenversand,Peter Franken,Berliner Platz 43,München,80805,Germany
4,39,Königlich Essen,Philip Cramer,Maubelstr. 90,Brandenburg,14776,Germany
5,44,Lehmanns Marktstand,Renate Messner,Magazinweg 7,Frankfurt a.M.,60528,Germany
6,52,Morgenstern Gesundkost,Alexander Feuer,Heerstr. 22,Leipzig,04179,Germany
7,56,Ottilies Käseladen,Henriette Pfalzheim,Mehrheimerstr. 369,Köln,50739,Germany
8,57,Paris spécialités,Marie Bertrand,265 boulevard Charonne,Paris,75012,France
9,63,QUICK-Stop,Horst Kloss,Taucherstraße 10,Cunewalde,01307,Germany


In [6]:
Query = """
SELECT * FROM OrderDetails
WHERE Quantity >= 40
AND
OrderID in (
            SELECT OrderID 
            FROM (
                    SELECT A.*, B.CustomerName 
                    FROM Orders AS A 
                    LEFT JOIN
                        (
                            SELECT * 
                            FROM Customers 
                            WHERE CustomerName like '%Ernst%' 
                            OR 
                            CustomerName like '%Stop%'
                        ) AS B
                    ON A.CustomerID=B.CustomerID
                )
            );
"""
df = pd.read_sql(Query,db)
df

In [9]:
Query = """
SELECT A.*, B.CustomerID, B.CustomerName 
FROM 
(
    SELECT * 
    FROM OrderDetails
    WHERE Quantity >= 40
) AS A
INNER JOIN
(
    SELECT C.*, D.CustomerName 
    FROM Orders AS C
    INNER JOIN
    (
        SELECT * 
        FROM Customers 
        WHERE CustomerName like '%Ernst%' 
        OR 
        CustomerName like '%Stop%'
    ) AS D
    ON C.CustomerID=D.CustomerID
) AS B
ON A.OrderID = B.OrderID
;
"""
df = pd.read_sql(Query,db)
df

,OrderDetailID,OrderID,ProductID,Quantity,CustomerID,CustomerName
0,30,10258,2,50,20,Ernst Handel
1,31,10258,5,65,20,Ernst Handel
2,44,10263,16,60,20,Ernst Handel
3,46,10263,30,60,20,Ernst Handel
4,277,10351,44,77,20,Ernst Handel
5,356,10382,33,60,20,Ernst Handel
6,357,10382,74,50,20,Ernst Handel
7,379,10390,31,60,20,Ernst Handel
8,380,10390,35,40,20,Ernst Handel
9,381,10390,46,45,20,Ernst Handel


In [11]:
Query = """
SELECT B.OrderID, A.CustomerID, A.Address, A.Country 
FROM Customers AS A INNER JOIN Orders AS B
ON A.CustomerID = B.CustomerID
;
"""
df = pd.read_sql(Query,db)
df

,OrderID,CustomerID,Address,Country
0,10308,2,Avda. de la Constitución 2222,Mexico
1,10365,3,Mataderos 2312,Mexico
2,10355,4,120 Hanover Sq.,UK
3,10383,4,120 Hanover Sq.,UK
4,10278,5,Berguvsvägen 8,Sweden
5,10280,5,Berguvsvägen 8,Sweden
6,10384,5,Berguvsvägen 8,Sweden
7,10265,7,24 place Kléber,France
8,10297,7,24 place Kléber,France
9,10360,7,24 place Kléber,France
